In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)
from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("mcc7_bnb_detsim_to_larcv_hires_v00_p00_out_0000.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()
iom.read_entry(10)

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("unit.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))
    img=iom.get_data(larcv.kProductImage2D,"tpc")
    im=larcv.as_ndarray(img.Image2DArray()[plane])
    plt.imshow(np.where(im>10.0,1.0,0.0).astype(np.uint8),cmap='Greys',interpolation='None')
    nz_pixels=np.where(im>0.0)
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.show()

In [ ]:
larbysimg.process(iom)

In [ ]:
pygeo   = geo2d.PyDraw()

mgr=larbysimg.Manager()
img_v = []

xranges=((300,900),(400,700),(450,700))
yranges=((700,1275),(750,900),(1350,1600))

plane=0
for mat in mgr.InputImages():
    #img_v.append(pygeo.image(mat).transpose())
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12))
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    plt.show()
    plane+=1

dm=mgr.DataManager()

In [ ]:
for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(20,20))

    #Get the image
    shape_img = img_v[plane]
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.max(nz_pixels[0])-10,np.min(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(900,1100)
    #ax.set_xlim(400,700)

    data=dm.Data(0)

    all_p0=data._all_ctors_v_v[plane]

    for ctor in all_p0:
        pts=np.array([[pt.x,pt.y] for pt in ctor])
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(2.0)
    ax.set_title("ALL Contours")
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(900,1100)
    #ax.set_xlim(400,700)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(1)

    split_lines=data._split_line_v_v[plane]

    for sline in split_lines:
        ax.plot([0,900],[sline.y(0),sline.y(900)])

    atomic_p0=data._atomic_ctor_v_v[plane]

    for atomic in atomic_p0:
        pts=np.array([[pt.x,pt.y] for pt in atomic])
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(2.0)
    ax.set_title("ALL Contours")
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

    cv=larbysimg.Manager().GetClusterAlg(2)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    cv_data=dm.Data(2)

    cv_ctor_lines_v=cv_data._ctor_lines_v_v_v[plane]
    cv_inter_pts_v=cv_data._ipoints_v_v_v[plane]
    ix=-1

    for cv_ctor_lines in cv_ctor_lines_v:
    
        ix+=1
        color=colormap[ix]
        
        for line in cv_ctor_lines:
            plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)

        for inter_pt in cv_inter_pts_v[ix]:
            plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
            
    ax=plt.gca()
    plt.show()

In [ ]:
import matplotlib.patches as patches
xranges=((600,700),(575,675),(575,700))
yranges=((900,1000),(775,875),(1375,1475))
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    #ax.set_ylim(np.min(nz_pixels[0])+50,np.max(nz_pixels[0])-40)
    #ax.set_xlim(np.min(nz_pixels[1])+250,np.max(nz_pixels[1])-90)
    #ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    #ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(875,975)
    #ax.set_xlim(600,700)
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    ix=-1

    for cv_ctor_lines in cv_ctor_lines_v:
    
        ix+=1
        color=colormap[ix]
        for line in cv_ctor_lines:
            pass#plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
        
        for inter_pt in cv_inter_pts_v[ix]:
            pass#plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        

    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    cv_data=dm.Data(3)
    circle_v = cv_data._circledata_v_v[plane]
    #for circle in circle_v:
        #c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',fc='none',lw=10)
        #ax.add_patch(c)

    ref_data = dm.Data(4)

    for idx in xrange(ref_data._xs_vv[plane].size()):
        pt = ref_data._xs_vv[plane][idx]
        plt.plot([pt.x],[pt.y],color='r',marker='o',markersize=20)
        print pt.x,pt.y
        #print ref_data._x_vv[0][idx],ref_data._y_vv[0][idx]
    for l in ref_data._pca_vv[plane]:
        if l.dir.y == 1:
            plt.plot([0,900],[0,2000],'-',color='orange',linewidth=2)
        else:
            plt.plot([0,900],[l.y(0),l.y(900)],'-',color='orange',linewidth=2)
        #print l.pt.x,l.pt.y,l.dir.x,l.dir.y    

    print ref_data._circle_trav_vv[plane].size()
    for circle in ref_data._circle_trav_vv[plane]:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.2,fc='none',lw=10)
        ax.add_patch(c)

    ax=plt.gca()
    ax.set_aspect(1.0)
    plt.savefig
    plt.show()

In [ ]:
import cv2

# pick ones closest in time
radius=15.
cx_v=[]
cy_v=[]
for plane in xrange(3):
    print 'plane',plane
    vtx   = ref_data._cand_vtx_v[plane]
    score = ref_data._cand_score_v[plane]
    print vtx.x, vtx.y, score
    cx_v.append(vtx.x)
    cy_v.append(vtx.y)

xranges=((600,700),(575,675),(575,700))
yranges=((900,1000),(775,875),(1375,1475))
#xranges=((875,975),(575,675),(575,700))
#yranges=((600,700),(775,875),(1375,1475))

#cx,cy = (641.995788574,936.084777832)
#cx,cy = (648.477050781,940.827514648)
#cx,cy = (642.,935.)
for plane in xrange(3):

    fig,ax = plt.subplots(figsize=(10,10))
    cx = cx_v[plane]
    cy = cy_v[plane]

    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    pimg=cv2.linearPolar(shape_img,(cx,cy),radius*2,flags=cv2.WARP_FILL_OUTLIERS)
    col = 0.5 * pimg.shape[1]
    xv=[]
    yv=[]

    row_coll_v=[[]]
    combine_last = False

    for row in xrange(pimg.shape[0]):
        if pimg[row][col] < 1: 
            if len(row_coll_v[-1]): row_coll_v.append([])
            continue
        row_coll_v[-1].append(row)
    
    for idx in xrange(len(row_coll_v)):
        row_coll_v[idx] = np.array(row_coll_v[idx])

    # Check if the last element should be merged or not
    if len(row_coll_v) > 1:
        if row_coll_v[0][0] == 0 and row_coll_v[-1][-1] == pimg.shape[0]-1:
            row_coll_v[-1] -= (pimg.shape[0] - 1)
            row_coll_v[0] = np.concatenate((row_coll_v[0],row_coll_v[-1]),axis=0)
            row_coll_v.pop()
        for row_coll in row_coll_v:

            ar = np.array(row_coll)
            if len(row_coll) < 1: continue
            ave_row = ar.mean()
            if ave_row < 0:
                ave_row += (pimg.shape[0]-1)
            xv.append(cx + radius * np.cos(2 * 3.1415 * float(ave_row/pimg.shape[0])))
            yv.append(cy + radius * np.sin(2 * 3.1415 * float(ave_row/pimg.shape[0])))
            plt.plot([col],[ave_row],marker='o',color='red',markersize=10)
    
    plt.imshow(pimg,cmap='Greys',interpolation='none')

    fig,ax = plt.subplots(figsize=(20,20))
    for idx in xrange(len(xv)):
        plt.plot([xv[idx]],[yv[idx]],marker='o',color='yellow', markersize=10)
        pt1 = Vector(DTYPE)(cx,cy)
        pt2 = Vector(DTYPE)(xv[idx],yv[idx])
        d = geo2d.dir(pt1,pt2)
        l = geo2d.HalfLine(DTYPE)(pt1,d)

        if d.x < 0:
            plt.plot([cx,0],[cy,l.y(0.)],marker='o',color='red',linestyle='--',linewidth=2)
        else:
            plt.plot([cx,shape_img.shape[1]],[cy,l.y(shape_img.shape[1])],marker='o',color='red',linestyle='--',linewidth=2)
    
    c=patches.Circle((cx,cy),radius,ec='cyan',fc='none',alpha=0.5,lw=10)
    ax.add_patch(c)

    plt.plot([cx],[cy],marker='o',color='red',markersize=10)
    
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    plt.imshow(shape_img,cmap='Greys',interpolation='none')


In [ ]:
for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(20,20))

    #Get the image
    shape_img = img_v[plane]
    shape_img=np.where(shape_img>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.max(nz_pixels[0])-10,np.min(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(900,1100)
    ax.set_xlim(400,700)

    data=dm.Data(5)

    all_ctor=data._ctor_vv[plane]

    for ctor in all_ctor:
        pts=[]
        for idx in xrange(ctor.size()): pts.append([ctor[idx].x,ctor[idx].y])
        pts = np.array(pts)
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(2.0)
    ax.set_title("ALL Contours")
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(900,1100)
    #ax.set_xlim(400,700)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(6)

    split_lines=data._split_line_v_v[plane]

    for sline in split_lines:
        ax.plot([0,900],[sline.y(0),sline.y(900)])

    atomic_p0=data._atomic_ctor_v_v[plane]

    for atomic in atomic_p0:
        pts=np.array([[pt.x,pt.y] for pt in atomic])
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(2.0)
    ax.set_title("ALL Contours")
    plt.show()